# Connect to ADLS

In [1]:
from azure.datalake.store import core
from azure.datalake.store import lib

token = lib.auth(tenant_id="bb9bc891-a81c-4296-bd5d-15aedd92502c",
                 client_id="d1b5de72-67ad-49ef-ab10-b76d2ca70be7",
                 client_secret="hoCO_19xn8frF~ivX81ioD.Yr5O9Pv9A-g",
                 resource="https://datalake.azure.net/")
adl = core.AzureDLFileSystem(token, store_name="cvetky")

# Common functions

In [2]:
import pandas as pd

def read_json_file(filename):
    with adl.open(filename) as f:
        return pd.read_json(f)

# Load pandas from JSON files on data lake

In [3]:
adl_statistics_directory = "/task_lm_system_data/"
files = sorted(adl.listdir(adl_statistics_directory), reverse=True)

frames = []
for filename in files:
    frames.append(read_json_file(filename))

df = pd.concat(frames)

# Calculate execution status

In [4]:
import numpy as np

zero_exit_code = df["exit_code"] == 0
not_killed = df["signal"] != "KILL"
df["status"] = np.where(zero_exit_code & not_killed, "SUCCESS", "FAILED")

# Cleanse data

In [5]:
df["last_3_output_lines"] = df["last_3_output_lines"].str.replace("(\s)+", " ", regex=True)
df["last_3_output_lines"] = df["last_3_output_lines"].str[-256:] # due to SAC limitation

# Upload statistics file

In [6]:
import gspread

gc = gspread.service_account(filename='./task_lm_credentions.json')

statistics_spreadsheet = gc.open_by_key('1pfA38C3QaxW-OV70JAypA5zkhNzPgzn9Kgowi01goyA')
statistics_worksheet = statistics_spreadsheet.worksheet('statistics') 
statistics_worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1pfA38C3QaxW-OV70JAypA5zkhNzPgzn9Kgowi01goyA',
 'updatedRange': 'statistics!A1:O73',
 'updatedRows': 73,
 'updatedColumns': 15,
 'updatedCells': 1095}